In [4]:
import os
from multiprocessing import cpu_count
import numpy as np
import shutil
import paddle
import paddle.fluid as fluid

In [5]:
s = [["我",1],["是",3],["谁",8]]
print(s)
d = dict(s)
print(d)
d['他'] = 5
d1 = {"你":2,"他":4}
d.update(d1) # 使用新字典里面的数据，更新原字典，若是原字典有的，则更新，若是没有的，则新增
print(d)

[['我', 1], ['是', 3], ['谁', 8]]
{'我': 1, '是': 3, '谁': 8}
{'我': 1, '是': 3, '谁': 8, '他': 4, '你': 2}


In [6]:
def create_data_list(data_root_path):
    with open(data_root_path + 'test_list.txt', 'w') as f:
        pass
    with open(data_root_path + 'train_list.txt', 'w') as f:
        pass

    with open(os.path.join(data_root_path, 'dict_txt.txt'), 'r', encoding='utf-8') as f_data:
        dict_txt = eval(f_data.readlines()[0]) # 将读取的数据（dictionary），
                                               # 并通过eval函数转为python的字典对象，eval将内容作为python语句执行，并返回执行结果

    with open(os.path.join(data_root_path, 'news_classify_data.txt'), 'r', encoding='utf-8') as f_data:
        lines = f_data.readlines() 
        i = 0
        for line in lines:
            title = line.split('_!_')[-1].replace('\n', '') # 使用 _!_ 分割字符串，并将末尾的换行符进行替换
            l = line.split('_!_')[1] # 获数据的类型
            labs = ""
            if i % 10 == 0: # 如果是当前数据的条数是10的整数倍，将其添加到测试数据集合
                with open(os.path.join(data_root_path, 'test_list.txt'), 'a', encoding='utf-8') as f_test:# join函数会根据当前操作系统的类型，自动生成文件链接的字符串
                    # 打开测试数据
                    for s in title:
                        lab = str(dict_txt[s]) # 获取标题的编码
                        labs = labs + lab + ',' # 将编码添加到编码的字符串中
                    labs = labs[:-1]
                    labs = labs + '\t' + l + '\n' # 编码\t类型\n
                    print(title + "\n" + labs)
                    f_test.write(labs)
            else:
                with open(os.path.join(data_root_path, 'train_list.txt'), 'a', encoding='utf-8') as f_train:
                    for s in title:
                        lab = str(dict_txt[s])
                        labs = labs + lab + ','
                    labs = labs[:-1]
                    labs = labs + '\t' + l + '\n' # 编码\t类型\n
                    f_train.write(labs)
            i += 1
    print("数据列表生成完成！")


# 把下载得数据生成一个字典
def create_dict(data_path, dict_path):
    dict_set = set()
    # 读取已经下载得数据
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    # 把数据生成一个集合
    for line in lines:
        title = line.split('_!_')[-1].replace('\n', '') # 取分割后的数组的最后一个
        #print(title)
        for s in title:
            dict_set.add(s) # 集合中的元素是不能重复的
    # 把集合转换成字典，一个字对应一个数字
    dict_list = []
    i = 0
    for s in dict_set:
        dict_list.append([s, i])
        #print([s,i])
        i += 1
    # 添加未知字符
    dict_txt = dict(dict_list)
    end_dict = {"<unk>": i}
    dict_txt.update(end_dict)
    # 把这些字典保存到本地中
    with open(dict_path, 'w', encoding='utf-8') as f:
        f.write(str(dict_txt))

    print("数据字典生成完成！")


# 获取字典的长度
def get_dict_len(dict_path):
    with open(dict_path, 'r', encoding='utf-8') as f:
        line = eval(f.readlines()[0])
    print(type(line))
    return len(line.keys())
